In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import json

In [94]:
!pip install python-firebase

In [162]:
def bmi_calc (height, weight):
  height=height/100
  bmi=weight/(height**2)
  return bmi


def reject_conditions (age, height, weight, alc_score, smoke_score, haemo=13):
  valid_user=1
  if (age<18 or age>60):
    valid_user=0
  if (bmi_calc(height, weight)>30):
    valid_user=0
  if alc_score==5 or smoke_score==5:
    valid_user=0
  if (haemo<13):
    valid_user=0

  return valid_user
  

In [153]:
#Helper functions

def age_quality(age):
  qa=0
  if (age>18 and age<23):
    qa-=1
  if(age>=45 and age<55):
    qa=-1.5
  if(age>=55 and age<60):
    qa=-3
  return qa

  

def bmi_checker(height, weight):
  qa=0
  bmi=bmi_calc(height, weight)
  if (bmi<18.5):
    qa=qa-(18.5-bmi)*1
  if (bmi>=18.5 and bmi<=25):
    qa=qa
  if (bmi>25):
    qa=qa-(bmi-25)

  return round(qa,2)


def past_diseases_checker(past_diseases):
  qa=0
  l=len(past_diseases)
  for d in range(l):
    if past_diseases[d]=='Diabetes' or past_diseases[d]=='Cardiac':
      qa-=5
    if past_diseases[d]=='Jaundice':
      qa-=3
    if past_diseases[d]=='Blood Pressure':
      qa=-2
  return qa


def tattoo_score(t):
  if t==True:
    return -4
  else:
    return 0


def toxic_subs(alc_s, smo_s):
  qa=0
  qa-= (alc_s*0.6 + smo_s*0.7)
  return round(qa,2)


def heart_fitness (avg_hr, haemo=14):
  qa=0
  if (haemo<13.8):
    qa=qa-1.5
  if (avg_hr<65 or avg_hr>95):
    qa-=2
  return qa



def fit_exer (sportsgym):
  if sportsgym==True:
    return 1
  else:
    return -1

def sleep_check(avg_sl):
  qa=0
  if (avg_sl<6):
    qa-=(6-avg_sl)
  if(avg_sl>=6 and avg_sl<=8):
    qa+=1
  if (avg_sl>8):
    qa-=(avg_sl-8)
  return qa












In [154]:
def quality_index (age, height, weight, past_diseases, tattoo_date, alc_score, smoke_score, sportsgym, average_sleep, heart_rate, haemo=14):
  health_score = 100
  age_qa=age_quality(age)
  bmi_qa= bmi_checker(height, weight)
  pd_qa= past_diseases_checker(past_diseases)
  tat_qa=tattoo_score(tattoo_date)
  alc_sm_qa=toxic_subs(alc_score, smoke_score)
  heart_qa=heart_fitness(heart_rate, haemo)
  spo_qa=fit_exer(sportsgym)
  slp_qa = sleep_check(average_sleep)
  health_score+=(age_qa+bmi_qa+pd_qa+tat_qa+alc_sm_qa+heart_qa+spo_qa+slp_qa)
  return health_score

In [155]:
quality_index (21, 170, 76, ['Jaundice'],False, 0, 0, 13.8, True, 5, 75)


  


86.7

In [156]:
def compute (c):
  age=c['age']
  alc_score=c['alcoholLevel']
  haemo= c['haemoglobin']
  pd=c['pastDiseases']
  h=c['height']
  w=c['weight']
  tat=c['tatoos']
  smo_score=c['smokingLevel']
  spogym=c['sportsAndGym']
  avg_sl=c['sleepLevel']
  heart_rate=c['heartRate']

  cond=reject_conditions (age, h, w, alc_score, smo_score, haemo=13)
  if(cond==0):
    return -1
  else:
    return quality_index (age, h, w, pd, tat, alc_score, smo_score, haemo, spogym, heart_rate)



  


In [157]:
from firebase import firebase


In [158]:

firebase = firebase.FirebaseApplication('https://bloodhive-27308.firebaseio.com/', None)

In [159]:
firebase.get ('/Manit/new_user/blood_group', None)

'O'

In [165]:
while True:
  check=firebase.get ('/Manit/new_user/status', None)
  if (check==1):
    blood_g=firebase.get('/Manit/new_user/blood_group', None)
    user_id=firebase.get('/Manit/new_user/user_id', None)
    path='/Manit/Donor/'+blood_g+'/'+user_id
    c=firebase.get(path, None)
    x=compute (c)
    firebase.put(path, 'qualityIndex', x)
    firebase.put('/Manit/new_user/', 'status', 0)




{'activeDonor': False,
 'age': 21,
 'alcoholLevel': 1,
 'bloodGroup': 'O',
 'email': 'sagarfootballer07@gmail.com',
 'gender': 'Male',
 'haemoglobin': 11,
 'heartRate': 76,
 'height': 178,
 'latAdd': '25.7825333',
 'longAdd': '84.4768173',
 'mobile': '9434484646',
 'ok': True,
 'pastDiseases': ['Diabetes'],
 'qualityIndex': 100,
 'sleepLevel': 10,
 'smokingLevel': 0,
 'sportsAndGym': False,
 'tatoos': False,
 'user_id': 'N0o7eiRh6QVXVNncntad9E5t0323',
 'user_name': 'Sagar',
 'weight': 89}